# Chapter 13: Functions III: Lambdas and functors

In [4]:
from random import randint

Okay, so you can define a function inside a function.

In [ ]:
def containedFun():
    def plus(a,b):
        return a+b
    print(plus(3,4))

But, now that we're outside *containedFun*'s scope, we can no longer access plus.

For mapping, we always defined an extra global function, even though we only used it in one place. It would be cleaner to only define that function where it is needed.

In [ ]:
def mapListBetter(lst):
    def plus2(a):
        return a+2
    return map(plus2, lst)

We see that most of the time, we will write

In [2]:
#def funName(<argList>):    
#    return <some simple expression>

Since this pattern is so common, there is a shorter form that allows in-lining.

I like these things so much, I am going to use a pretty box to introduce them:

In [3]:
{():(),():(),():()}
{}###############{}
{}# INTRODUCING #{}
{}#   LAMBDAS   #{}
{}###############{}
{():(),():(),():()}
#The following three functions are exactly equivalent:
def withDef():
    def plus(a,b):
        return a+b
    print(plus(3,4))

def setLambda():
    plus = lambda a,b : a+b
    print(plus(3,4))

def inlineLambda():
    print ( (lambda a,b : a+b) (3,4))

Okay, so the second one wasn't too bad, right?

You can see that lambda is, so to speak, the part of the function that "does the work".

You can read "*lambda x : f(x)*" as "*a function that takes one parameter and returns f of that parameter*."

In [ ]:
#"plus = lambda a,b : a+b" therefore, reads as
#"'plus' is defined to be a function that takes two parameters and adds them."
# \plus/ \------- = -----/ \---lambda----/\----a,b-----------/\-:-/\--a+b--/

Clear on the second one? If not, ask now!

Now we get to the third one. Let's go way back. Like, way, way, back.

In [ ]:
def useVariables():
    #This operation:
    a = 7
    print(a + 4)
    #is equivalent to this one:
    print(7 + 4)

Okay, you remember that. Formally, anywhere a literal (like 7) appears, we can use a name for that literal (here, we used a). The EXACT same thing is true for functions. Lambdas, then, are the literal functions!

In the third case, I have skipped the step of defining the name plus, instead I just used the lambda where I would have normally used a name.

This is CRAZY. Is your mind sufficiently blown yet? I remember mine was!

We use lambdas frequently with map and filter.

Previously, we defined a global function. At the beginning of this chapter, I introduced local functions. Now, we skip all of that with lambdas. 

In [ ]:
def lambdaListProc():
    a = [1, 2, 3, 4, 5, 6, 7]
    print(list(filter((lambda x : x % 2 == 0), a)))
    #(I have enclosed the lambda in parentheses for clarity.)
    print(list(map (lambda x : x + 3, a)))
    #and here I haven't.

Okay, so that's all well and good. Let me first give a few LIMITATIONS OF LAMBDAS:

A lambda may contain exactly one expression. It is invalid, therefore, to do

In [ ]:
#lambda x : if x > 2:
#      3
#   else:
#      4

because it is more than an expression. (It's technically a statement.)

You cannot assign in a lambda:

In [ ]:
#lambda x : y = x

is invalid.

Basically, remember that the lambda returns what is after the colon; if you wouldn't return something from a function, don't put it in a lambda.

Remember, there is a single-expression *if* expression, as was discussed in Chapter 05.

It's mostly syntactic sugar, but it allows for things like

In [ ]:
#lambda x: (int(x) if isNumeric(x) else -1)
#(This would be equivalent to the following:
#def funName(x):
#   if isNumeric(x):
#       return int(x)
#   else:
#       return -1
#)

How are lambdas better than global functions? I hope you're convinced they're terser, but they (and local functions) can also be *returned from functions*.

This is awesome:

In [ ]:
def addNum(n):
    return lambda x : n + x

def useAddNum():
    plus3 = addNum(3)
    #plus3 is now a function that adds three to a given number. 
    print(plus3(4))

Okay, to be honest, you can return global functions, too, but this is usually useless, because the function you're returning the functor to already had access to it because it was global in the first place. 

Here's a function that returns a function that strips bad letters from a nucleic acid strand. If you specify it's DNA, it allows AGTC, if you specify it's RNA, it allows AGUC. 

In [ ]:
def processNA(dOrR):
    def isValid(letter):
        
        if dOrR == 'D':
            allowLets = ['A', 'G', 'C', 'T']
        elif dOrR == 'R':
            allowLets = ['A', 'G', 'C', 'U']
        else:
            print ("Invalid dOrR selection: " + dOrR)
            a = 1/0 #Throw an unrecoverable error. 
        return letter in allowLets
    return isValid

def useProcess():
    junkyNA = "AGTUTCTGGCTAUTCATGATUGTCUTGACTUGUCGTAUCGATCUTGTCAGTCATUGUCAT"
    fixedList = list(filter( processNA('D'), junkyNA))
    print(fixedList)

Basically, this is not very useful at the scale we're talking about. But say you have a very complex function you're using to filter a list, and you occasionally need to tweak some factors of the function. Using this style allows a programmer to tweak the inner workings of a function without having to pick through it to see how it works.

I should formalize something here: Scope. Uh, get ready.

When you start defining a function, you open up a new "scope". This is a space for storing variables in the environment. Outside of a scope, the variables from that scope are inaccessible. We already know this:

In [ ]:
def bug():
    def a():
        x = 1
    def b():
        return x
    a()
    return b()

*x* was defined in *a*(), so it's not available in *b*. Similarly, I can't run *a*() from outside bug, because it's no longer in scope.

What's weird is that a nested function includes its enclosing scope.

In [ ]:
def notBug():
    x = 1
    def a():
        return x
    return a()

When I call *a*() in this function, the scope inside *a* includes its parent's scope, so it has access to it. 

In Python (not in all languages), you can't write to an external scope:

In [ ]:
def maybeBug():
    x = 1
    def a():
        x = 2
        print(x)
    a()
    print(x)

This is really peculiar. Python sees that you're trying to assign to a variable in an outer scope, and that's not allowed, so it creates a variable named *x* in the inner function's scope. This *x* now "shadows" the *x* from external scope. This only is a problem when I'm trying to assign to the external scope.

Okay, you say, I'll make it really clear that I want to assign to the outer scope:

In [ ]:
def bug2():
    x = 1
    def a():
        x = x+1
    a()
    return x

Okay, what? Here, we're probing the semantics of the interpreter-

When the interpreter encounters the line *x*=*x*+1, first it creates an (empty) entry in the local scope for *x*, then when it gets to *x*+1, it sees that *x* is not bound in the local scope, so it's an error.

In [ ]:
def bug3():
    x = 1
    def a():
        b = x + 1
        x = b
    a()
    return x

Okay, this is even stranger: It turns out that Python looks ahead to see what names will be defined in the local scope and makes room for them before it starts running code in the scope.

This is mostly us seeing some of the assumptions that they made as they were writing the Python interpreter. By deciding what scope a variable refers to before the code is executed, we can spend less time looking up the value associated with a variable's name. 

Phew. Deep breath. If you don't get the scope business, you'll be fine. 

It comes up just about never. If you're having scope problems, your code is  probably badly written. If you use the same symbol for different variables in different places, your code is badly written and you should fix that. 

Even if you do get the complexities of scoping, your colleagues probably don't. =)

Okay, now we're just getting esoteric:

Because *addA*() tries to mutate a variable in an external scope, it doesn't work.

In [ ]:
#def iWish():
#   a = 1
#   def addA():
#       a = a+1
#   def getA():
#       return a
#   return (addA, getA)

If it did, we would have a hidden variable - there'd be no way to access it, other than to use those functions.

To get this effect, we must do something a bit more abtruse...

The main difference in strange (below) is that a was just an integer above, but now it's a list on the heap. 

In [ ]:
def strange():
    a = [1]
    def mutA():
        a[0] = a[0]+1
    def viewA():
        return a[0]
    return (mutA, viewA)

def useStrange():
    mut8, view = strange()
    mut8()
    print(view())

What's happening here is really peculiar: Even after we return from *strange*, its scope "lives on" in *mutA* and *viewA*. Even though we don't have access to *a* from outside *strange*(), methods defined inside it "close" over their parents' scopes, so they retain a copy of this information. I did this without telling you in *processNA* and *addNum*: Note that the inner function refers to arguments that the outer function got, even after the inner function is returned and we've left the outer function. 

Oh, the reason we can seemingly mutate something in the outer scope is that we don't *reassign* *a* from the outer scope. Remember back to Chapter 08, a list is stored on the heap by providing the addresses of its contents.

Here, we change the data inside the list, but the list itself remains in the same place in memory.

####   *Exercises*

1 - Write a function that returns a lambda that checks divisibility:

In [ ]:
#divisibleLamb(n) -> function(x)

 that checks if *x* is divisible by *n*.

2 - Explain what's happening here:

In [ ]:
def sumGen(y):
    return lambda x:(y if x==0 else sumGen(x+y))
#Usage examples:
#sumGen(2)(0) -> 2
#sumGen(3)(5)(6)(0)-> 14

3 - *hammingDistances* from Chapter 09 can be elegantly expressed with two maps:
Each row is a map of the hamming distance from one sequence to every other one and the whole table is a row for every sequence.

Write a function that generates this table using only *map* to iterate over the sequences. (you can use *hammingDistance* from Chapter 07 to get the distance between two sequences). There should be no iteration constructs (*for* or *while*) and the function may not recur.